# CP321 Final Project

In [298]:
import pandas as pd
import numpy as np
import plotly.express as px


In [299]:
datafile = "/Users/jashan/Desktop/CP321-FinalProject/data/finalData.csv"

In [300]:
raw_data = pd.read_csv(datafile)
raw_data

,REF_DATE,GEO,DGUID,Age (2),"Highest certificate, diploma or degree (16)",Major field of study - Classification of Instructional Programs (CIP) 2021 (500),Occupation - Unit group - National Occupational Classification (NOC) 2021 (821A),Gender (3),UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Men+,NaN,0,units,0,NaN,2.2.2.2.3.2,12025,NaN,NaN,NaN,0
1,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Women+,NaN,0,units,0,NaN,2.2.2.2.3.3,7755,NaN,NaN,NaN,0
2,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",0 Legislative and senior management occupations,Women+,NaN,0,units,0,NaN,2.2.2.2.4.3,10,NaN,NaN,NaN,0
3,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",00 Legislative and senior managers,Women+,NaN,0,units,0,NaN,2.2.2.2.5.3,10,NaN,NaN,NaN,0
4,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",000 Legislative and senior managers,Women+,NaN,0,units,0,NaN,2.2.2.2.6.3,10,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130827,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9 Occupations in manufacturing and utilities,Men+,NaN,0,units,0,NaN,14.2.4.476.725.2,10,NaN,NaN,NaN,0
130828,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",90 Middle management occupations in manufactur...,Men+,NaN,0,units,0,NaN,14.2.4.476.726.2,10,NaN,NaN,NaN,0
130829,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",900 Middle management occupations in manufactu...,Men+,NaN,0,units,0,NaN,14.2.4.476.727.2,10,NaN,NaN,NaN,0
130830,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9001 Managers in manufacturing and utilities,Men+,NaN,0,units,0,NaN,14.2.4.476.728.2,10,NaN,NaN,NaN,0


In [ ]:
redundant_cols = [
    "DGUID",
    "Age (2)",
    "UOM",
    "UOM_ID",
    "SCALAR_FACTOR",
    "SCALAR_ID",
    "STATUS",
    "SYMBOL",
    "TERMINATED",
    "DECIMALS",
    "VECTOR",
    "REF_DATE",
    "COORDINATE",
]
master_df = raw_data.drop(columns=redundant_cols)
master_df.columns = 
master_df

,GEO,EDUCATION,MAJOR,OCCUPATION,GENDER,COUNT
0,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Men+,12025
1,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Women+,7755
2,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",0 Legislative and senior management occupations,Women+,10
3,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",00 Legislative and senior managers,Women+,10
4,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",000 Legislative and senior managers,Women+,10
...,...,...,...,...,...,...
130827,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9 Occupations in manufacturing and utilities,Men+,10
130828,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",90 Middle management occupations in manufactur...,Men+,10
130829,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",900 Middle management occupations in manufactu...,Men+,10
130830,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9001 Managers in manufacturing and utilities,Men+,10


In [302]:
master_df.isnull().sum()

GEO           0
EDUCATION     0
MAJOR         0
OCCUPATION    0
GENDER        0
COUNT         0
dtype: int64

Fix Gender

In [303]:
def clean_gender_COUNTs(COUNT):
    return COUNT[:-1]


def get_noc_codes(COUNT):
    noc = COUNT.split(" ")[0]
    if not noc.isdigit():
        return np.nan
    return noc

def get_hierarchy_number(COUNT):
    if type(COUNT) is float:
        return 0
    return len(COUNT)

def clean_occupations(COUNT):
    COUNT = COUNT.split(" ")
    COUNT.pop(0)
    return " ".join(COUNT)


master_df["GENDER"] = master_df.GENDER.apply(clean_gender_COUNTs) # remove plus sign form gender COUNTs
master_df["NOC"] = master_df.OCCUPATION.apply(get_noc_codes) # Split NOC codes based on categories.
master_df["Hierarchy"] = master_df.NOC.apply(get_hierarchy_number)
master_df["OCCUPATION_C"] = master_df.OCCUPATION.apply(clean_occupations)

master_df

,GEO,EDUCATION,MAJOR,OCCUPATION,GENDER,COUNT,NOC,Hierarchy,OCCUPATION_C
0,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Men,12025,NaN,0,occupations
1,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Women,7755,NaN,0,occupations
2,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",0 Legislative and senior management occupations,Women,10,0,1,Legislative and senior management occupations
3,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",00 Legislative and senior managers,Women,10,00,2,Legislative and senior managers
4,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",000 Legislative and senior managers,Women,10,000,3,Legislative and senior managers
...,...,...,...,...,...,...,...,...,...
130827,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9 Occupations in manufacturing and utilities,Men,10,9,1,Occupations in manufacturing and utilities
130828,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",90 Middle management occupations in manufactur...,Men,10,90,2,Middle management occupations in manufacturing...
130829,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",900 Middle management occupations in manufactu...,Men,10,900,3,Middle management occupations in manufacturing...
130830,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9001 Managers in manufacturing and utilities,Men,10,9001,4,Managers in manufacturing and utilities


In [304]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130832 entries, 0 to 130831
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   GEO           130832 non-null  object
 1   EDUCATION     130832 non-null  object
 2   MAJOR         130832 non-null  object
 3   OCCUPATION    130832 non-null  object
 4   GENDER        130832 non-null  object
 5   COUNT         130832 non-null  int64 
 6   NOC           130477 non-null  object
 7   Hierarchy     130832 non-null  int64 
 8   OCCUPATION_C  130832 non-null  object
dtypes: int64(2), object(7)
memory usage: 9.0+ MB


In [305]:
master_df.GEO.unique()

array(['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories',
       'Nunavut'], dtype=object)

In [306]:
broader_cats = master_df[master_df.Hierarchy==1].OCCUPATION_C.unique()
broader_cats

array(['Legislative and senior management occupations',
       'Business, finance and administration occupations',
       'Natural and applied sciences and related occupations',
       'Health occupations',
       'Occupations in education, law and social, community and government services',
       'Occupations in art, culture, recreation and sport',
       'Sales and service occupations',
       'Trades, transport and equipment operators and related occupations',
       'Natural resources, agriculture and related production occupations',
       'Occupations in manufacturing and utilities'], dtype=object)

In [307]:
temp_df = master_df[master_df.Hierarchy == 1].groupby(["GEO","OCCUPATION_C","GENDER"],as_index=False).COUNT.sum()
temp_df

,GEO,OCCUPATION_C,GENDER,COUNT
0,Alberta,"Business, finance and administration occupations",Men,98320
1,Alberta,"Business, finance and administration occupations",Women,248605
2,Alberta,Health occupations,Men,31680
3,Alberta,Health occupations,Women,140075
4,Alberta,Legislative and senior management occupations,Men,15080
...,...,...,...,...
255,Yukon,Occupations in manufacturing and utilities,Women,40
256,Yukon,Sales and service occupations,Men,1615
257,Yukon,Sales and service occupations,Women,1880
258,Yukon,"Trades, transport and equipment operators and ...",Men,3435


In [308]:
alberta = temp_df[temp_df.GEO == "Alberta"].copy()
alberta

,GEO,OCCUPATION_C,GENDER,COUNT
0,Alberta,"Business, finance and administration occupations",Men,98320
1,Alberta,"Business, finance and administration occupations",Women,248605
2,Alberta,Health occupations,Men,31680
3,Alberta,Health occupations,Women,140075
4,Alberta,Legislative and senior management occupations,Men,15080
5,Alberta,Legislative and senior management occupations,Women,6370
6,Alberta,Natural and applied sciences and related occup...,Men,132840
7,Alberta,Natural and applied sciences and related occup...,Women,41165
8,Alberta,"Natural resources, agriculture and related pro...",Men,57725
9,Alberta,"Natural resources, agriculture and related pro...",Women,15175


In [309]:
fig = px.bar(
    alberta,
    x="OCCUPATION_C",
    y="COUNT",
    color="GENDER",
    barmode="group",
    title="Gender Distribution Across Occupations in Alberta",
    labels={"OCCUPATION_C": "Occupation Code", "Count": "Number of People"},
)

fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor="white",
    hovermode="x unified",
    legend_title_text="Gender",
)

fig.update_traces(
    textfont_size=12, textangle=0, textposition="outside", cliponaxis=False
)

fig

In [310]:
industry = temp_df[
    temp_df.OCCUPATION_C == "Business, finance and administration occupations"
].copy()
industry

,GEO,OCCUPATION_C,GENDER,COUNT
0,Alberta,"Business, finance and administration occupations",Men,98320
1,Alberta,"Business, finance and administration occupations",Women,248605
20,British Columbia,"Business, finance and administration occupations",Men,121755
21,British Columbia,"Business, finance and administration occupations",Women,287075
40,Manitoba,"Business, finance and administration occupations",Men,28670
41,Manitoba,"Business, finance and administration occupations",Women,67945
60,New Brunswick,"Business, finance and administration occupations",Men,14880
61,New Brunswick,"Business, finance and administration occupations",Women,39110
80,Newfoundland and Labrador,"Business, finance and administration occupations",Men,7625
81,Newfoundland and Labrador,"Business, finance and administration occupations",Women,22885


FINAL HA

In [311]:
fig = px.bar(
    industry,
    x="GEO",
    y="COUNT",
    color="GENDER",
    barmode="group",
    log_y=True,
    template="plotly_white",
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
)

fig.update_layout(xaxis={"categoryorder": "total descending"})

fig.update_layout(
    xaxis_tickangle=-30,
    xaxis=dict(
        categoryorder="total descending",
        tickangle=-30,
        title="<b>Administrative Units</b>",
        tickfont=dict(size=12),
        title_font=dict(size=12 ),
    ),
    yaxis=dict(
        title="<b>Number of People (Log Scale)</b>",
        tickfont=dict(size=12),
        title_font=dict(size=12),
        showgrid=True,
        gridcolor="rgba(200, 200, 200, 0.3)",
    ),
    height=600,
    margin=dict(l=50, r=50, t=80, b=100),
)

# update Tile
fig.update_layout(
    title=dict(
        text="<b>Gender Composition of  Aerospace Engineering Departments Across Canada</b>",
        font=dict(size=20, family="Arial", color="#2a4978"),
    )
)
fig.update_layout(
    legend=dict(
        orientation="h",
        title=None,
        y=1,
        x=1,
        xanchor="right",
        yanchor="bottom",
    ),
    legend_bordercolor="black",
    legend_borderwidth=1,
)
fig.update_traces(
    textfont_size=10,
    textangle=0,
    textposition="outside",
    cliponaxis=False,
    texttemplate="%{y:.2s}",
    hovertemplate="<b>Region:</b> %{x}<br>"
    "<b>Count:</b> %{y}<extra></extra>",
)


fig

In [312]:
cat_5_df = master_df[master_df.Hierarchy == 5].groupby(["GEO", "OCCUPATION_C"],as_index=False).COUNT.sum()
cat_5_df

,GEO,OCCUPATION_C,COUNT
0,Alberta,Accommodation service managers,3040
1,Alberta,"Accommodation, travel, tourism and related ser...",930
2,Alberta,Accounting and related clerks,18610
3,Alberta,Accounting technicians and bookkeepers,17935
4,Alberta,"Actors, comedians and circus performers",735
...,...,...,...
5625,Yukon,Utilities managers,20
5626,Yukon,Veterinarians,20
5627,Yukon,Water and waste treatment plant operators,50
5628,Yukon,Web developers and programmers,15


In [313]:
cat_5_df.OCCUPATION_C.unique()

array(['Accommodation service managers',
       'Accommodation, travel, tourism and related services supervisors',
       'Accounting and related clerks',
       'Accounting technicians and bookkeepers',
       'Actors, comedians and circus performers',
       'Administrative assistants', 'Administrative officers',
       'Administrators - post-secondary education and vocational training',
       'Advertising, marketing and public relations managers',
       'Aerospace engineers', 'Agricultural and fish products inspectors',
       'Agricultural representatives, consultants and specialists',
       'Agricultural service contractors and farm supervisors',
       'Air pilots, flight engineers and flying instructors',
       'Air traffic controllers and related occupations',
       'Air transport ramp attendants',
       'Aircraft assemblers and aircraft assembly inspectors',
       'Aircraft instrument, electrical and avionics mechanics, technicians and inspectors',
       'Aircraft mech

In [314]:
ESSENTIAL_OCCUPATIONS = {"nurse", "police", "fire"}


def get_essential_occupations(occupations):
    essentail_cols = []
    for occupation in occupations:
        lower_occupation = occupation.lower()
        if (
            any(essential in lower_occupation for essential in ESSENTIAL_OCCUPATIONS)
            and "nursery" not in lower_occupation
        ):
            essentail_cols.append(occupation)
    return essentail_cols


required_cols = get_essential_occupations(cat_5_df.OCCUPATION_C.unique())

In [315]:
essential_df = cat_5_df[cat_5_df["OCCUPATION_C"].isin(required_cols)]
essential_df

,GEO,OCCUPATION_C,COUNT
79,Alberta,Commissioned police officers and related occup...,90
162,Alberta,Fire chiefs and senior firefighting officers,305
163,Alberta,Firefighters,4760
252,Alberta,Licensed practical nurses,8820
300,Alberta,"Nurse aides, orderlies and patient service ass...",33445
...,...,...,...
5463,Yukon,Firefighters,35
5507,Yukon,Licensed practical nurses,75
5529,Yukon,"Nurse aides, orderlies and patient service ass...",175
5556,Yukon,Police officers (except commissioned),140


In [316]:
fig = px.line_polar(
    essential_df,
    r="COUNT",
    theta="OCCUPATION_C",
    color="GEO",
    line_close=True,
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    log_r=True,
    height=600
)
# update Tile
fig.update_layout(
    title=dict(
        text="<b>Gender Composition of  Aerospace Engineering Departments Across Canada</b>",
        font=dict(size=20, family="Arial", color="#2a4978"),
    )
)


fig

In [317]:
fig = px.line_polar(
    essential_df,
    r="COUNT",
    theta="OCCUPATION_C",
    color="GEO",
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
    template="plotly_white",
    labels={"COUNT": "Count (log)", "OCCUPATION_C": "Occupation Category"},
    hover_name="GEO",
    hover_data={"COUNT": ":.2f"},
    log_r=True,
    start_angle=30,
    line_close=True,
)

# update Tile
fig.update_layout(
    title=dict(
        text="<b>Gender Composition of  Aerospace Engineering Departments Across Canada</b>",
        font=dict(size=20, family="Arial", color="#2a4978"),
    )
)

# Additional styling
fig.update_layout(
    font=dict(family="Arial", size=12),
    polar=dict(
        radialaxis=dict(
            angle=45, showline=True, linewidth=2, gridcolor="rgba(10,10,10,0.2)"
        ),
        angularaxis=dict(linewidth=2, gridcolor="rgba(10,10,10,0.2)"),
    ),
    height=800,
    margin=dict(l=100, r=100, t=80, b=20),
)
fig.update_layout(
    legend=dict(
        orientation="h",
        title=None,
        y=-.2,
        x=1,
        xanchor="right",
        yanchor="bottom",
    ),
    legend_bordercolor="black",
    legend_borderwidth=1,
)

fig

In [318]:
import plotly.express as px

# Create the polar line chart
fig = px.line_polar(
    essential_df,
    r="COUNT",
    theta="OCCUPATION_C",
    color="GEO",
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
    template="plotly_white",
    labels={"COUNT": "Count (log)", "OCCUPATION_C": "Occupation Category"},
    hover_name="GEO",
    hover_data={"COUNT": ":.2f"},
    log_r=True,
    start_angle=30,
    line_close=True,
    line_shape="spline",
)


fig.update_layout(
    title=dict(
        text="<b>Gender Composition of Aerospace Engineering Departments Across Canada</b>",
        font=dict(size=22, family="Arial", color="#2a4978"),
        x=0.5, 
        xanchor="center",
    ),
    font=dict(family="Arial", size=13, color="#333"),
    polar=dict(
        radialaxis=dict(
            angle=45,
            showline=True,
            linewidth=2,
            gridcolor="rgba(10,10,10,0.2)",
            tickangle=0,
        ),
        angularaxis=dict(
            linewidth=2,
            gridcolor="rgba(10,10,10,0.2)",
            tickfont=dict(size=12),
        ),
    ),
    legend=dict(
        orientation="h",
        title=None,
        y=-0.15,  # Adjust position to avoid overlap
        x=0.5,
        xanchor="center",
        yanchor="bottom",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    height=600,
    margin=dict(l=80, r=80, t=90, b=50),  # Adjusted margins for balance
)

fig

In [319]:
essential_df

,GEO,OCCUPATION_C,COUNT
79,Alberta,Commissioned police officers and related occup...,90
162,Alberta,Fire chiefs and senior firefighting officers,305
163,Alberta,Firefighters,4760
252,Alberta,Licensed practical nurses,8820
300,Alberta,"Nurse aides, orderlies and patient service ass...",33445
...,...,...,...
5463,Yukon,Firefighters,35
5507,Yukon,Licensed practical nurses,75
5529,Yukon,"Nurse aides, orderlies and patient service ass...",175
5556,Yukon,Police officers (except commissioned),140


In [320]:
essential_df_opc = essential_df[essential_df.OCCUPATION_C == "Firefighters"]
essential_df_opc

,GEO,OCCUPATION_C,COUNT
163,Alberta,Firefighters,4760
673,British Columbia,Firefighters,4665
1182,Manitoba,Firefighters,1060
1681,New Brunswick,Firefighters,620
2158,Newfoundland and Labrador,Firefighters,330
2568,Northwest Territories,Firefighters,50
2911,Nova Scotia,Firefighters,820
3324,Nunavut,Firefighters,10
3626,Ontario,Firefighters,12755
4525,Quebec,Firefighters,6360


In [321]:
import plotly.express as px

fig = px.pie(
    essential_df_opc,
    values="COUNT",
    names="GEO",
    title="Geographical Distribution",
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
    hole=0.3,
    labels={"COUNT": "Count"},
    template="plotly_white",
)

fig.update_layout(
    font_family="Helvetica Neue", 
    title_font_size=26,
    margin=dict(t=50, b=10, l=10, r=10),
    showlegend=True,
    legend_title_text="Regions",
    legend_font_size=12,
    legend_title_font_size=14,
    height = 600,
    width=800
)

fig.update_layout(
    legend=dict(
        orientation="v",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
)

fig.update_traces(
    textposition="inside",
    textinfo="percent+label",
    insidetextfont=dict(color="black", size=14, family="Arial"),
    marker=dict(line=dict(color="white", width=2)),  # More distinct slice separation
    hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>",
    pull=[
        0.1 if i == essential_df_opc["COUNT"].idxmax() else 0.02
        for i in range(len(essential_df_opc))
    ],  # Highlight largest slice
)

fig.show()

In [322]:
ENGINEERING_OCCUPATIONS = {
    "electrical and electronics engineers",
    "mechanical engineers",
    "computer engineers",
}


def get_engineering_occupations(occupations):
    essentail_cols = []
    for occupation in occupations:
        lower_occupation = occupation.lower()
        if (any(eng in lower_occupation for eng in ENGINEERING_OCCUPATIONS)):
            essentail_cols.append(occupation)
    return essentail_cols


engineering_occupations = get_engineering_occupations(cat_5_df.OCCUPATION_C.unique())
engineering_occupations

['Computer engineers (except software engineers and designers)',
 'Electrical and electronics engineers',
 'Mechanical engineers']

In [323]:
eng_df= master_df[master_df.OCCUPATION_C.isin(engineering_occupations)]
eng_df

,GEO,EDUCATION,MAJOR,OCCUPATION,GENDER,COUNT,NOC,Hierarchy,OCCUPATION_C
3654,Newfoundland and Labrador,"Postsecondary certificate, diploma or degree","Business, management and public administration",21301 Mechanical engineers,Men,25,21301,5,Mechanical engineers
3656,Newfoundland and Labrador,"Postsecondary certificate, diploma or degree","Business, management and public administration",21310 Electrical and electronics engineers,Men,15,21310,5,Electrical and electronics engineers
3657,Newfoundland and Labrador,"Postsecondary certificate, diploma or degree","Business, management and public administration",21311 Computer engineers (except software engi...,Men,10,21311,5,Computer engineers (except software engineers ...
4419,Newfoundland and Labrador,"Postsecondary certificate, diploma or degree",Physical and life sciences and technologies,21301 Mechanical engineers,Men,10,21301,5,Mechanical engineers
4421,Newfoundland and Labrador,"Postsecondary certificate, diploma or degree",Physical and life sciences and technologies,21310 Electrical and electronics engineers,Men,10,21310,5,Electrical and electronics engineers
...,...,...,...,...,...,...,...,...,...
124942,Yukon,"Postsecondary certificate, diploma or degree","Architecture, engineering, and related trades",21301 Mechanical engineers,Men,10,21301,5,Mechanical engineers
124944,Yukon,"Postsecondary certificate, diploma or degree","Architecture, engineering, and related trades",21310 Electrical and electronics engineers,Men,10,21310,5,Electrical and electronics engineers
127837,Northwest Territories,"Postsecondary certificate, diploma or degree","Architecture, engineering, and related trades",21301 Mechanical engineers,Men,40,21301,5,Mechanical engineers
127839,Northwest Territories,"Postsecondary certificate, diploma or degree","Architecture, engineering, and related trades",21310 Electrical and electronics engineers,Men,15,21310,5,Electrical and electronics engineers


In [ ]:
def update_education(value):
    if value == "Postsecondary certificate, diploma or degree":
        return "With Postsecondary Education"
    return "Without Postsecondary Education"

eng_df["EDUCATION"] = eng_df["EDUCATION"].apply(update_education)

/var/folders/pl/0r4mcs9s32l3wwcm1j4_s7nm0000gn/T/ipykernel_39665/83796719.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [325]:
px.treemap(
    eng_df,
    values="COUNT",
    color="GEO",
    path=["GEO", "OCCUPATION_C", "EDUCATION", "MAJOR"],
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
)

In [ ]:
import plotly.express as px
import numpy as np

# Compute midpoint dynamically for balanced color scale
midpoint = np.median(eng_df["COUNT"])

# Create treemap with refined styling
fig = px.treemap(
    eng_df,
    path=[px.Constant("Canada"), "GEO", "OCCUPATION_C", "EDUCATION"],
    values="COUNT",
    color="OCCUPATION_C",
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
    color_continuous_midpoint=midpoint,
    height=850,
    template="plotly_white",
)

# Update layout for a high-end professional presentation
fig.update_layout(
    margin=dict(t=0, l=0, r=0, b=0),
    hoverlabel=dict(bgcolor="white", font_size=14, font_family="Helvetica"),
    uniformtext=dict(minsize=10,mode="hide"),
)


fig.update_traces(
    root_color="lightgrey",
    textinfo="label+value+percent parent",
    texttemplate="<b>%{label}</b><br>(%{percentParent:.1%})",
    textposition="middle left",
    marker=dict(line=dict(width=.5, color="black")), 
    hovertemplate="<b>%{label}</b><br>Count: %{value:,}<br>%{percentParent:.1%} of parent<extra></extra>",
    textfont_size=15, 
)

fig.show()

In [327]:
master_df

,GEO,EDUCATION,MAJOR,OCCUPATION,GENDER,COUNT,NOC,Hierarchy,OCCUPATION_C
0,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Men,12025,NaN,0,occupations
1,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Women,7755,NaN,0,occupations
2,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",0 Legislative and senior management occupations,Women,10,0,1,Legislative and senior management occupations
3,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",00 Legislative and senior managers,Women,10,00,2,Legislative and senior managers
4,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",000 Legislative and senior managers,Women,10,000,3,Legislative and senior managers
...,...,...,...,...,...,...,...,...,...
130827,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9 Occupations in manufacturing and utilities,Men,10,9,1,Occupations in manufacturing and utilities
130828,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",90 Middle management occupations in manufactur...,Men,10,90,2,Middle management occupations in manufacturing...
130829,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",900 Middle management occupations in manufactu...,Men,10,900,3,Middle management occupations in manufacturing...
130830,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9001 Managers in manufacturing and utilities,Men,10,9001,4,Managers in manufacturing and utilities


In [332]:
major_df = master_df.groupby(["MAJOR"],as_index=False).COUNT.sum()
major_df

,MAJOR,COUNT
0,"Agriculture, natural resources and conservation",1696505
1,"Architecture, engineering, and related trades",14156625
2,"Business, management and public administration",15592435
3,Education,3942630
4,Health and related fields,9702475
5,Humanities,3330980
6,"Mathematics, computer and information sciences",3561390
7,"No postsecondary certificate, diploma or degree",28591650
8,Other,16855
9,"Personal, protective and transportation services",3992715


In [ ]:
fig = px.bar(major_df,x="MAJOR",y="COUNT")
fig.show()

In [370]:
import plotly.express as px

fig = px.bar_polar(
    major_df,
    theta="MAJOR",
    r="COUNT",
    log_r=True,
    # Visual enhancements:
    color="COUNT",  # Color bars by count
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
    hover_name="MAJOR",
    hover_data={"COUNT": ":.0f"},  # Format count display
    labels={
        "COUNT": "Number of Professionals",
    },
    width=900,
    
)

# Further customization:
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, showticklabels=True),
        angularaxis=dict(showticklabels=True, tickfont=dict(size=12)),
    ),
    font=dict(family="Arial", size=12),
    coloraxis_colorbar=dict(title="Count"),
    title_font=dict(size=20),
    margin=dict(t=50, b=50, l=50, r=50),
)

fig.show()

In [358]:
import plotly.express as px

# Create horizontal bar chart
fig = px.bar(
    major_df,
    y="MAJOR",  # Categories on y-axis for horizontal bars
    x="COUNT",  # Values on x-axis
    orientation="h",  # Make it horizontal
    log_x=True,  # Keep the log scale
    color="COUNT",
    color_discrete_sequence=px.colors.qualitative.Pastel2_r,
    text_auto=".2s",
    labels={"COUNT": "Number of Professoinal (log scale)", "MAJOR": "Major"},
    height=800,
)

# Further customization:
fig.update_layout(
    template="plotly_white",
    font=dict(family="Arial", size=12),
    yaxis=dict(
        title=None,
        categoryorder="total ascending",  # Sort bars by value
        tickfont=dict(size=12),
        tickangle=-30,
    ),
    xaxis=dict(
        title="Number of Professional In Field (log scale)",
        tickprefix="10^",  # Indicate log scale
        showgrid=True,
    ),
    coloraxis_colorbar=dict(title="Count"),
    title_font=dict(size=20),
    uniformtext_minsize=8,  # Minimum size for text labels
    margin=dict(l=0, r=0, t=0, b=0),  # Left margin for long major names
)

# Improve bar text visibility
fig.update_traces(
    textfont_size=12, textposition="outside", cliponaxis=False  # Prevent text clipping
)

fig.show()

In [371]:
education_df = master_df.groupby(["GEO","EDUCATION","MAJOR"],as_index=False).COUNT.sum()
education_df

,GEO,EDUCATION,MAJOR,COUNT
0,Alberta,High (secondary) school diploma or equivalency...,"No postsecondary certificate, diploma or degree",2839925
1,Alberta,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",921155
2,Alberta,"Postsecondary certificate, diploma or degree","Agriculture, natural resources and conservation",220035
3,Alberta,"Postsecondary certificate, diploma or degree","Architecture, engineering, and related trades",1983015
4,Alberta,"Postsecondary certificate, diploma or degree","Business, management and public administration",1689520
...,...,...,...,...
173,Yukon,"Postsecondary certificate, diploma or degree","Mathematics, computer and information sciences",1930
174,Yukon,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",4805
175,Yukon,"Postsecondary certificate, diploma or degree",Physical and life sciences and technologies,4040
176,Yukon,"Postsecondary certificate, diploma or degree",Social and behavioural sciences and law,11035
